In [ ]:
import os
import veeringCV

from importlib import reload
reload(veeringCV)

In [ ]:
importDirectory = '/Users/sean/mbp_storage/251017/portMain'
exportDirectory = '/Users/sean/mbp_storage/250622/portMain/test_out'
importPath = os.path.join(importDirectory,'Port Main'+'.h5')

In [ ]:
## Load H5 File
normalisations = [1,3]
features = veeringCV.VeeringNormalisation(importPath)
features.RunNormalisation(normalisations)

In [ ]:
## Generate Pixcels
targetColour = [256,165,130]

thresholds = veeringCV.Thresholding(features.pixcels,targetColour,[180,205,5])
thresholds.Run_Thresholding()
thresholds.Generate_Stripe_Pixcels(int(0),features.pixcels,features.origShape)
thresholds.Count_Filter(float(0.1),float(0.9))
thresholds.Stripes_Plot_Clean()


In [ ]:
## Thresholds Graph
thresholds.Threshold_Graphs()


In [ ]:
## Offset Sweep
thresholds.Threshold_Sweep_Graphs([-6,-4,-2,0,2], features.origShape, features.pixcels)

In [ ]:
thresholds.Generate_Stripe_Pixcels(-2,features.pixcels,features.origShape)

In [ ]:
## Count Cleaning Graphs
thresholds.Cnt_Filter_Plot()

In [ ]:
multipliers = [0.005, 0.001]
countFilter_ind = thresholds.countFilter_ind
clusterOffset = 0
set_db = veeringCV.Set_DB(thresholds.stripes, multipliers, countFilter_ind, clusterOffset)

In [ ]:
set_db.Set_PCA_DB()

In [ ]:
set_db.Cluster_Filter_FIG()

In [ ]:
set_db.Cluster_Plot_FIG()

In [ ]:
stripes, setPCA = set_db.Make_DB_Scan_Set()

In [ ]:
len(set_db.toKeep)


In [ ]:
pic_db = veeringCV.Pic_DB(stripes,set_db.setRotation_degrees, setPCA)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.scatter(stripes[np.where(stripes[:,2]==0)[0],0],stripes[np.where(stripes[:,2]==0)[0],1])

In [ ]:
pic_db.Pic_DB_Scan(True,25,0.01,.0001)

In [ ]:
cp_dict = pic_db.clusterPoints_dict

In [ ]:
fit_calc = veeringCV.Fit_Spline_Calc(cp_dict, features.origShape, setPCA)
fit_calc.Fit_Splines(False, 0.1, 5, 0.1)
fit_calc.Link_Splines(False, 75, 1.25, 4)
fit_calc.Filter_Splines(4)
fit_calc.Calc_Properties()

In [ ]:
fit_calc.stripe_properties[40]

In [ ]:
import numpy as np
picture = veeringCV.VeeringNormalisation(importPath)
picture.RunNormalisation([0])
textStart = picture.origShape[0] + 30
picture = np.reshape(picture.pixcels, picture.origShape, order='F')

In [ ]:
import matplotlib.pyplot as plt
splines_set = fit_calc.splines_set_final
stripe_properties = fit_calc.stripe_properties
stripeLables = ['1/4', '1/2', '3/4', '7/8']
imagePlots, axes = plt.subplots(len(splines_set.keys()),1 , figsize=(180,180))

for i in range(len(splines_set.keys())):
    pic = list(splines_set.keys())[i]
    plot_points_stripe = stripe_properties[pic][0]
    plot_points_ant = stripe_properties[pic][1]
    stripeStats = stripe_properties[pic][2]

    axes[i].imshow(picture[:,:,:,pic].astype('uint8'))
    for points_orig in plot_points_stripe:
        axes[i].plot(points_orig[:,1],points_orig[:,0],color='lime', linestyle='-', linewidth=2)

        axes[i].set_title(pic)
        axes[i].set_xticks([])
        axes[i].set_yticks([])
    for plotAnnotations in plot_points_ant:
        axes[i].plot(plotAnnotations[2:4,1],plotAnnotations[2:4,0] ,color='red', linestyle='-', linewidth=1)
        axes[i].plot(plotAnnotations[4:6,1],plotAnnotations[4:6,0],color='blue', linestyle='-', linewidth=1)
        axes[i].plot(plotAnnotations[0:2,1],plotAnnotations[0:2,0],color='yellow', linestyle='-', linewidth=1)

    axes[i].text(50,textStart,"Stripe", fontsize = 100)
    axes[i].text(150,textStart,"Draft")
    axes[i].text(250,textStart,"Camber")
    axes[i].text(350,textStart,"Front")
    axes[i].text(450,textStart,"Back")

    offset = 20
    for n in range(len(stripeStats)):
        axes[i].text(50,textStart+offset,stripeLables[n])
        axes[i].text(150,textStart+offset,round(stripeStats[n][0]*100,2))
        axes[i].text(250,textStart+offset,round(stripeStats[n][1]*100,2))
        axes[i].text(350,textStart+offset,round(stripeStats[n][2]*100,2))
        axes[i].text(450,textStart+offset,round(stripeStats[n][3]*100,2))
        offset += 15